In [1]:
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from networks import Encoder, Decoder, MetaModel, TrainingCallback
import numpy as np
import ast
from sindy import SINDy
import os


In [3]:
encoder = Encoder()

In [8]:
momo=tf.keras.models.load_model("/data/uab-giq/scratch/matias/sandra/networks/run1/encoder_511")

In [10]:
momo.optimizer.get_weights()

[array([[ 0.02505166, -0.00093502, -0.00132424, ...,  0.00157893,
         -0.00052265, -0.01164991],
        [ 0.02381385, -0.00091414, -0.00155163, ...,  0.0014801 ,
         -0.00038985, -0.01255735],
        [ 0.02259202, -0.00088746, -0.00174407, ...,  0.00138649,
         -0.00027406, -0.01324608],
        ...,
        [ 0.02415363, -0.00127656, -0.00382656, ...,  0.0007644 ,
          0.0004292 , -0.02799806],
        [ 0.02626561, -0.00157117, -0.00440273, ...,  0.00073944,
          0.00056695, -0.03190333],
        [ 0.02848556, -0.0018913 , -0.00503439, ...,  0.00070553,
          0.00072053, -0.03614683]], dtype=float32),
 array([ 4.7923002e-02, -1.2349673e-03,  3.5498317e-04,  8.6058062e-03,
        -4.3915692e-03,  1.0989758e-02,  1.1411887e-03,  1.1131480e-02,
         4.8104245e-03,  3.3198182e-02,  4.2569544e-02, -5.6336145e-03,
         5.5808118e-03,  4.1053323e-03,  2.4232522e-03, -2.8113496e-02,
        -6.6826460e-03, -3.5260110e-03,  3.0546598e-02, -1.6806303e-02

In [3]:
sindy1=SINDy()
sindy1.load_weights("/data/uab-giq/scratch/matias/sandra/networks/run0/sindy_11000/")

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /data/uab-giq/scratch/matias/sandra/networks/run0/sindy_11000/

In [5]:
sindy1.coeffs

<tf.Variable 'sin_dy/dense/kernel:0:0' shape=(3, 27) dtype=float32, numpy=
array([[ 0.        , -0.2874051 ,  0.        , -0.35773182,  0.        ,
         0.19659008,  0.        ,  0.1521247 , -0.11801123,  0.7033265 ,
         0.16338713,  0.5814898 ,  0.13949075,  0.23118523,  0.        ,
         0.        , -0.11038066, -0.19735841,  0.        ,  0.38824803,
         0.3213863 ,  0.33641833, -0.30027488, -0.30669263, -0.1034919 ,
         0.17556998,  0.        ],
       [-0.72970283, -0.39467755,  0.        ,  0.6563016 ,  0.        ,
         0.10190756,  0.        ,  0.24105406,  0.        , -0.3855203 ,
         0.10014518,  0.23272444,  0.29047665,  0.43499675,  0.29084662,
         0.        ,  0.        , -0.16534267,  0.        ,  0.24607953,
         0.10626309,  0.18738425,  0.        ,  0.10424263,  0.        ,
         0.        , -0.10207617],
       [ 0.        ,  0.46672985, -0.17201279, -0.3105065 ,  0.20803712,
         0.71167344,  0.        ,  0.        , -0.10

In [117]:
encoder=Encoder()
encoder.load_weights("/data/uab-giq/scratch/matias/sandra/networks/run0/encoder_10000/")

In [118]:
dencoder=Decoder()
dencoder.load_weights("/data/uab-giq/scratch/matias/sandra/networks/run0/decoder_10000/")

In [108]:
s=np.squeeze(sindy1.coeffs)
len(np.where(np.squeeze(np.reshape(s,[-1,1]))!=0)[0])

8

In [94]:
di = {}
c=0
#th.append(tf.reduce_prod(tf.stack([tf.pow(z[:,0], i),tf.pow(z[:,1], j),tf.pow(z[:,2], k)]), axis=0))
for i in range(3):
    for j in range(3):
        for k in range(3):
            di[str([i,j,k])] = c%27
            c+=1
di

{'[0, 0, 0]': 0,
 '[0, 0, 1]': 1,
 '[0, 0, 2]': 2,
 '[0, 1, 0]': 3,
 '[0, 1, 1]': 4,
 '[0, 1, 2]': 5,
 '[0, 2, 0]': 6,
 '[0, 2, 1]': 7,
 '[0, 2, 2]': 8,
 '[1, 0, 0]': 9,
 '[1, 0, 1]': 10,
 '[1, 0, 2]': 11,
 '[1, 1, 0]': 12,
 '[1, 1, 1]': 13,
 '[1, 1, 2]': 14,
 '[1, 2, 0]': 15,
 '[1, 2, 1]': 16,
 '[1, 2, 2]': 17,
 '[2, 0, 0]': 18,
 '[2, 0, 1]': 19,
 '[2, 0, 2]': 20,
 '[2, 1, 0]': 21,
 '[2, 1, 1]': 22,
 '[2, 1, 2]': 23,
 '[2, 2, 0]': 24,
 '[2, 2, 1]': 25,
 '[2, 2, 2]': 26}

In [109]:
np.squeeze(sindy1.coeffs[0])

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.68683684,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        ], dtype=float32)

In [112]:
np.where(np.squeeze(sindy1.coeffs[2])!=0)[0]

array([0, 1, 3, 9])

In [46]:
x = sindy1.coeffs
sindy1.set_weights(sindy1.get_weights())
#tf.where( tf.abs(x) > 0.1, x, 0)

In [57]:
sindy1 = SINDy()
sindy1(tf.random.uniform((1,128)))
x=sindy1.get_weights()[0]
y = tf.where(tf.abs(x)>0.1,x,0)
sindy1.set_weights([y])

In [12]:
decoder(tf.random.uniform((1,129)))
decoder.weights

[<tf.Variable 'decoder/dense/kernel:0' shape=(129, 32) dtype=float32, numpy=
 array([[ 0.00651381,  0.01154741,  0.01565262, ..., -0.01187098,
          0.00117404,  0.00445091],
        [-0.00449749, -0.01292601,  0.00087418, ..., -0.00469507,
         -0.00784563, -0.0135037 ],
        [ 0.01065787, -0.01166525, -0.01800974, ..., -0.00056845,
          0.01059807,  0.01707367],
        ...,
        [-0.00450859,  0.00648362,  0.00088653, ...,  0.00587128,
          0.01585008, -0.00111794],
        [ 0.0108447 , -0.01220426,  0.00606966, ...,  0.00012872,
         -0.00638299, -0.01648249],
        [ 0.00132619,  0.01854318, -0.01659602, ...,  0.01801863,
         -0.01677597, -0.00527157]], dtype=float32)>,
 <tf.Variable 'decoder/dense/bias:0' shape=(32,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)>,
 <tf.Variable 'decoder/dense_1/kernel:0'

In [23]:
if runni>0:
    indices = tf.range(start=0, limit=tf.shape(x)[0], dtype=tf.int32)
    idx = tf.random.shuffle(indices)
    x_data = tf.gather(x, idx)
    x_dot_data = tf.gather(x_dot, idx)
else:
    x_data = x
    x_dot_data = x_dot

In [5]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [8]:
with mirrored_strategy.scope():

    encoder = Encoder()
    decoder = Decoder()
    sindy = SINDy()
    models = [encoder, decoder, sindy]

    metamodel = MetaModel(models, total_epochs=400, when_zero_lambda3=1000)
metamodel.compile_models()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [9]:
history = metamodel.fit(x=x, y=x_dot, epochs=50, batch_size=25,
                callbacks=[TrainingCallback(), tf.keras.callbacks.TensorBoard("logs/prueba3"),
                              ])

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 6 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 1 all-reduces with algorithm = nccl, num_packs = 1


...

INFO:tensorflow:Summary name epoch_Total Loss is illegal; using epoch_Total_Loss instead.


Epoch 19/50
40/40 [==============================] - 1s 32ms/step - Total Loss: 0.2132 - Loss_0: 0.2130 - Loss_1: 1.8935e-04 - Loss_2: 0.0000e+00 - Loss_3: 1.0655e-05
INFO:tensorflow:Summary name epoch_Total Loss is illegal; using epoch_Total_Loss instead.


INFO:tensorflow:Summary name epoch_Total Loss is illegal; using epoch_Total_Loss instead.


Epoch 20/50
28/40 [====================>.........] - ETA: 0s - Total Loss: 0.0267 - Loss_0: 0.0266 - Loss_1: 6.5916e-05 - Loss_2: 0.0000e+00 - Loss_3: 1.0559e-05

KeyboardInterrupt: 

In [36]:
sindy.load_weights("nets/sindy_0/")

In [37]:
sindy.weights

[<tf.Tensor: shape=(3, 27), dtype=float32, numpy=
 array([[-0.9311571 ,  1.3730997 , -0.18049856, -0.23604348, -0.8861915 ,
          0.34579983,  0.        ,  0.3817444 ,  0.6329095 , -0.46864775,
          0.20121433,  0.        ,  0.        ,  1.4880794 ,  0.8682103 ,
         -0.7623767 ,  0.76911217,  0.        , -2.235712  ,  1.3080964 ,
          1.2098081 , -1.1070364 , -1.3574041 , -1.7704686 , -0.71796733,
         -0.5128483 ,  1.8050104 ],
        [-0.65252423,  0.17881115,  1.0176535 , -0.22862421,  1.2786188 ,
         -0.45869717, -0.42945144, -0.2683546 ,  1.3455132 , -0.28287098,
         -0.6332829 ,  0.5585188 , -0.8899651 ,  2.2761014 , -0.33276758,
         -1.0422409 , -0.13713206, -1.0998034 ,  0.        , -0.87539   ,
          0.5636082 ,  1.5775213 , -1.6637301 ,  0.59779716, -0.6056043 ,
          1.464332  ,  0.92218494],
        [-1.5582777 ,  1.0950025 ,  2.142236  , -0.66036254,  0.66531783,
          0.5450642 ,  0.39311463, -2.4012308 ,  1.6103411 ,  1.

In [63]:
sindy = SINDy()
sindy.load_weights("nets/sindy_11/")

ValueError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on nets/sindy_11/: Not found: nets/sindy_11; No such file or directory

In [18]:
from sindy import SINDy
sindy1 = SINDy()

In [20]:
sindy1.name

'sin_dy_1'

In [9]:
coeffs = tf.Variable(np.ones((3,27)).astype(np.float32), trainable=True, name="sindy/dense/kernel:0")
coeffs

<tf.Variable 'Variable:0' shape=(3, 27) dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

In [58]:
sindy.weigths

[<tf.Variable 'Variable:0' shape=(3, 27) dtype=float32, numpy=
 array([[-2.48601614e-03,  8.35985005e-01, -1.08248937e+00,
         -1.25054210e-01,  2.19034106e-01, -1.66024411e+00,
         -1.19687581e+00, -4.92233634e-02,  8.97786766e-02,
          3.82149875e-01,  9.09135304e-03,  2.49032676e-01,
          7.79314060e-03,  5.54208532e-02, -8.83790329e-02,
         -3.96450646e-02,  1.44127405e+00, -2.65611023e-01,
          1.15535867e+00, -1.99022663e+00,  2.95111537e-01,
          4.73870263e-02, -8.94377083e-02, -3.42485398e-01,
          7.68711209e-01, -7.77051866e-01, -3.71877491e-01],
        [ 6.50211632e-01, -8.26581493e-02,  1.02030349e+00,
          7.19511807e-02, -5.15373468e-01,  3.45013499e-01,
         -1.21267819e+00,  1.25622785e+00, -1.47071671e+00,
         -9.12182629e-02,  5.38204551e-01,  7.74763823e-02,
          1.82412457e+00,  1.05103528e+00,  1.50815874e-01,
         -2.12940145e+00,  5.59756756e-01,  1.24723963e-01,
          3.04355519e-04,  1.0376338